# **Optimización de XGBoost con Optuna**

**Introducción**

Entrenar y optimizar un modelo XGBoost para predecir precios de viviendas usando el dataset California Housing de scikit-learn. Se utiliza Optuna para encontrar los mejores hiperparámetros que minimicen el error cuadrático medio (RMSE).

# 1. Importación de librerias

In [ ]:
# Importar las bibliotecas necesarias
# -------------------------------------------
import numpy as np
import optuna
import xgboost as xgb
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [11]:
# Opcional: Desactivar la verificacion SSL por si se produce un error SSLCERTIFICATE_VERIFY_FAILED al
# descargar los datos
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
    getattr(ssl, '_create_unverified_context', None)): 
    ssl._create_default_https_context = ssl._create_unverified_context

# 2. Carga y procesamiento de datos
Se utiliza el dataset California Housing de sklearn y se divide en conjuntos de entrenamiento y prueba.

In [12]:
data = fetch_california_housing(as_frame=False)
X, y = data.data, data.target

# Dividir el dataset en 80% entrenamiento y 20% prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 3. Optimizar los hiperparámetros usando Optuna.


In [14]:
def objective(trial):
    '''Esta función evalúa los hiperparámetros sugeridos por Optuna
    utilizando la métrica de error cuadrático medio (RMSE).'''
    # Hiperparámetros a optimizar
    params = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
    }

    # Crear el modelo con los parámetros sugeridos
    model = xgb.XGBRegressor(**params, random_state=42)

    # Evaluar desempeño con validacion cruzada
    score = cross_val_score(model, X_train, y_train, scoring='neg_root_mean_squared_error', n_jobs=-1, cv=5)
    rmse = score.mean()
    return rmse

# 4. Crear un estudio con Optuna

In [15]:
# Se utiliza Optuna para encontrar los mejores hiperparámetros con la opcion de maximize
# ya que la metrica con la que se puntua en la validacion cruzada es neg_root_mean_squared_error
# el cual es negativo y maximizar un valor negativo es lo mismo que minimizar un valor positivo
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2024-11-28 22:06:35,316] A new study created in memory with name: no-name-3df16cb9-2840-4869-bfaf-7487274868f5
[I 2024-11-28 22:06:35,909] Trial 0 finished with value: -0.4983423439316109 and parameters: {'n_estimators': 450, 'max_depth': 8, 'learning_rate': 0.20568447518768634, 'subsample': 0.8960182330592356, 'colsample_bytree': 0.9248943075705749, 'gamma': 3.122724147933542, 'min_child_weight': 1}. Best is trial 0 with value: -0.4983423439316109.
[I 2024-11-28 22:06:36,173] Trial 1 finished with value: -0.501808203506544 and parameters: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.14836816078019133, 'subsample': 0.8930808657436098, 'colsample_bytree': 0.6439174530591841, 'gamma': 2.4038234693481684, 'min_child_weight': 7}. Best is trial 0 with value: -0.4983423439316109.
[I 2024-11-28 22:06:36,474] Trial 2 finished with value: -0.5051717310609414 and parameters: {'n_estimators': 204, 'max_depth': 4, 'learning_rate': 0.15247856264624426, 'subsample': 0.5752611161836119

FrozenTrial(number=61, state=1, values=[-0.4533989526933633], datetime_start=datetime.datetime(2024, 11, 28, 22, 7, 39, 737643), datetime_complete=datetime.datetime(2024, 11, 28, 22, 7, 42, 438709), params={'n_estimators': 447, 'max_depth': 8, 'learning_rate': 0.04073077701338349, 'subsample': 0.6976107290816179, 'colsample_bytree': 0.6895856810437626, 'gamma': 0.06932982127405926, 'min_child_weight': 2}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=500, log=False, low=50, step=1), 'max_depth': IntDistribution(high=10, log=False, low=1, step=1), 'learning_rate': FloatDistribution(high=0.3, log=False, low=0.01, step=None), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'gamma': FloatDistribution(high=10.0, log=False, low=0.0, step=None), 'min_child_weight': IntDistribution(high=10, log=False, low=1, step=1)}, trial_id=61

# 5. Resultados de la optimización

In [16]:
# Paso 6: Mostrar los mejores resultados
# ----------------------------------------
# Optuna ofrece funciones para obtener el mejor conjunto de parámetros.
print('Mejores hiperparámetros encontrados:')
print(study.best_params)

print('\nMejor RMSE obtenido:')
print(study.best_value)


Mejores hiperparámetros encontrados:
{'n_estimators': 447, 'max_depth': 8, 'learning_rate': 0.04073077701338349, 'subsample': 0.6976107290816179, 'colsample_bytree': 0.6895856810437626, 'gamma': 0.06932982127405926, 'min_child_weight': 2}

Mejor RMSE obtenido:
-0.4533989526933633


# 6. Entrenamiento del modelo final

In [17]:
# Entrenar el modelo final con los mejores hiperparámetros
# ----------------------------------------------------------------
best_params = study.best_params
final_model = xgb.XGBRegressor(**best_params, random_state=42)
final_model.fit(X_train, y_train)

# Predecir y evaluar el modelo final
final_preds = final_model.predict(X_test)
final_rmse = root_mean_squared_error(y_test, final_preds)

print('RMSE del modelo final:')
print(final_rmse)

RMSE del modelo final:
0.44430813722035


In [18]:
# Evaluar el desempeño del modelo final con MAPE (Interpretabilidad)
mean_absolute_percentage_error(y_test, final_preds)

np.float64(0.16713269328139715)

In [24]:
# Evaluar con MAE
mean_absolute_error(y_test, final_preds)

np.float64(0.28963875091084906)